Install packages

In [6]:
pip install requests wget bs4

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=e8880f9039ba2c159cb5cedff9b18eb8b4d20fcd7f48e2523a2335fb50cd9680
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [7]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import wget
import re
from datetime import date
import pandas as pd
import time
from time import time, sleep

current_date = date.today().strftime("%d/%m/%Y")

Get link to each state page

In [8]:
base_url = 'https://vidyutpravah.in/'
base_page = requests.get(base_url)
soup = BeautifulSoup(base_page.content, 'html.parser')
state = soup.find_all('div', class_='outer_div')

In [9]:
def get_state_link(i):
    beginning = 'https://vidyutpravah.in'
    p = state[i].find_all('a')
    paragraphs = []
    for x in p:
        paragraphs.append(str(x))
    if len(paragraphs) > 0:    
      final_href = paragraphs[0].split("=")[2].split('>')[0].split('"')[1]
      return ''.join([beginning, final_href])
    else:
      return ''.join([beginning, ''])

Get each state name

In [10]:
def get_state_name(i):
  p = state[i].find_all('a')
  paragraphs = []
  for x in p:
    paragraphs.append(str(x))
  if len(paragraphs) > 0:
    x=paragraphs[0].split(">")[1].split("<")[0].strip()
    return x

In [11]:
def get_list(function):
  my_list = []
  for i in range(15,49):
      my_list.append(function(i))
  return my_list


Get data on energy 

In [12]:
def get_data_list(link_list):
      Data_list = []
      for i in range(len(link_list)):
          link = link_list[i]
          state_name = get_state_name(i+15)

          if state_name != None and link != 'https://vidyutpravah.in': 
            base_url_3 = link
            base_page3 = requests.get(base_url_3)
            soup3 = BeautifulSoup(base_page3.content, 'html.parser')
            
            yest_demand_met = soup3.find_all('span', class_='value_PrevDemandMET_en value_StateDetails_en')
            current_demand = soup3.find_all('span', class_='value_DemandMET_en value_StateDetails_en')
            shortage_dur_peak = soup3.find_all('span', class_='value_PeakDemand_en value_StateDetails_en')
            shortage_energy = soup3.find_all('span', class_='value_TotalEnergy_en value_StateDetails_en')
            period = soup3.findAll('b')[0].text
            appending_list = [state_name,
                              (' ').join(yest_demand_met[0].get_text().strip().split('\xa0')),
                              (' ').join(current_demand[0].get_text().strip().split('\xa0')),
                              (' ').join(shortage_dur_peak[0].get_text().strip().split('\xa0')),
                              (' ').join(shortage_energy[0].get_text().strip().split('\xa0')),
                              period,
                              current_date]
            Data_list.append(appending_list)
      return Data_list  

Convert to dataframe

In [13]:
final_df = pd.DataFrame(columns = ['State',"Yesterday State's Demand Met","Current State's Demand Met","Shortage for yesterday during peak","Shortage for yesterday energy",'Time','Date'])

Make our code to run automatically (5 tomes insted of for because someone can start code like in 12:59, an to get 4 full periods will need 5 iterations

In [ ]:
for i in range(5):
    start = time()
    link_list = get_list(get_state_link)
    data_list = get_data_list(link_list)
    append_df = pd.DataFrame(data_list, columns = ['State',"Yesterday State's Demand Met","Current State's Demand Met","Shortage for yesterday during peak","Shortage for yesterday energy",'Time','Date'])
    final_df = pd.concat([final_df,append_df])
    final_df.reset_index(drop = True, inplace = True)
    final_df.to_excel('energy_scrapping_final.xlsx')
    stop = time()
    sleep( max(900 - (stop-start),0) )
